# Creation of Gold Product Tables


In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, bb93e72f-855e-47b3-996b-fd2cb61b2030, 3, Finished, Available, Finished)

In [ ]:
#creation of table schema

In [2]:
%%sql
create table if not exists ecom_proj_LH.gold_product
(
    Product_ID long,
    Product_Category string,
    Product string,
    Created_TS timestamp,
    Modified_TS timestamp
)
using DELTA

StatementMeta(, bb93e72f-855e-47b3-996b-fd2cb61b2030, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
#creating max_date variable
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from ecom_proj_LH.gold_product").first()[0]

StatementMeta(, bb93e72f-855e-47b3-996b-fd2cb61b2030, 9, Finished, Available, Finished)

In [12]:
df_bronze=spark.sql("""select distinct 
           Product_Category,
           Product
           from ecom_proj_LH.bronze_sales where Modified_TS>'{}'""".format(Max_Date))

StatementMeta(, bb93e72f-855e-47b3-996b-fd2cb61b2030, 14, Finished, Available, Finished)

In [13]:
#creating Product_ID variable
Max_ID = spark.sql("select coalesce(max(Product_ID),0) from ecom_proj_LH.gold_product").first()[0]

StatementMeta(, bb93e72f-855e-47b3-996b-fd2cb61b2030, 15, Finished, Available, Finished)

In [14]:
df_final = df_bronze.withColumn("Product_ID",monotonically_increasing_id()+Max_ID+1)

StatementMeta(, bb93e72f-855e-47b3-996b-fd2cb61b2030, 16, Finished, Available, Finished)

In [15]:
#created a view called View product
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, bb93e72f-855e-47b3-996b-fd2cb61b2030, 17, Finished, Available, Finished)

In [ ]:
# Merging the data into gold_product table

In [17]:
%%sql
merge into ecom_proj_LH.gold_product as gp
using ViewProduct as vp
on gp.Product = vp.Product and gp.Product_Category = vp.Product_Category
when matched then 
update SET 
gp.Modified_TS = CURRENT_TIMESTAMP()

when not matched then 
insert(
    gp.Product_ID,
    gp.Product_Category,
    gp.Product,
    gp.Created_TS,
    gp.Modified_TS
)
VALUES
(
    vp.Product_ID,
    vp.Product_Category,
    vp.Product,
    CURRENT_TIMESTAMP(),
    CURRENT_TIMESTAMP()
)

StatementMeta(, bb93e72f-855e-47b3-996b-fd2cb61b2030, 19, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>